## Package Imports

In [11]:
import sys
from pathlib import Path
import os
import numpy as np

In [2]:
project_working_dir = str(Path(sys.path[0]).parent)
sys.path += [project_working_dir]
os.chdir(project_working_dir)

## Local Imports

In [3]:
from src.utils.read_obj import parse_obj
from src.utils.file_io import read_json
from src.simulation.setup.extract_clothing_vertex_data import extract_all_piece_vertices
from src.parameters import AVATAR_SCALING

In [4]:
avatar_mesh = parse_obj('./assets/BodyMesh.obj', './assets/BodyAnnotations.json')
avatar_mesh.scale_vertices(AVATAR_SCALING)

clothing_data = read_json('./assets/sewing_shirt.json')
all_pieces, sewing = extract_all_piece_vertices(clothing_data, avatar_mesh)

## Profile cpu version

In [5]:
trimesh = avatar_mesh.trimesh

In [6]:
vertices = all_pieces["L-1"].mesh.vertices_3d

In [7]:
%timeit trimesh.contains(vertices)

2.37 s ± 446 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit trimesh.nearest.on_surface(vertices)

618 ms ± 87.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit all_pieces["L-1"].mesh.offset_vertices((0, 0, 0))

299 µs ± 19.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Save numpy vertices

In [24]:
def save_numpy_array(filename: str, arr: np.ndarray):
    np.save(f'{project_working_dir}/profiling/numpy/{filename}', arr)

In [37]:
vertices.shape

(3205, 3)

In [25]:
save_numpy_array('piece_vertices.npy', vertices)

In [26]:
len(set(trimesh.faces.flatten())) / len(trimesh.faces.flatten())

0.16700542005420055

In [27]:
len(trimesh.faces.flatten())

5904

In [23]:
triangle_vertices = np.array([
    [vertices[i], vertices[j], vertices[k]] for i, j, k in trimesh.faces
], dtype=np.float32)

In [28]:
triangle_vertices.shape

(1968, 3, 3)

In [29]:
save_numpy_array('triangle_vertices.npy', triangle_vertices)

In [34]:
save_numpy_array('triangle_normals.npy', trimesh.face_normals)

In [36]:
trimesh.face_normals.shape

(1968, 3)